### Import Libraries

In [1]:
import great_expectations as gx
import pandas as pd

### Creating GX Context 

In [2]:
context = gx.get_context(mode="ephemeral")


### Creating DataSource

In [3]:
data_source_name = 'sample_peaks_300'
data_source = context.data_sources.add_pandas(name=data_source_name)

### Creating DataAsset


In [4]:
data_asset_name = 'Peak_entity_asset'
data_asset = data_source.add_dataframe_asset(name=data_asset_name)

### Creating BatchDefinition

In [5]:
batch_definition_name = 'full_batch'
batch_definition = data_asset.add_batch_definition_whole_dataframe(batch_definition_name)

### Creating Expectations

In [6]:
expectation_id = gx.expectations.ExpectColumnValuesToBeUnique(column="id")
expectation_id_visibility = gx.expectations.ExpectColumnValuesToNotBeNull(
    column="id"
)

In [7]:
expectation_name_value = gx.expectations.ExpectColumnDistinctValuesToBeInSet(
    column="name",
    value_set=["Dhaulagiri", "Everest", "Annapurna", "Gasherbrum", "K2", "Lhotse", "Makalu", "Manaslu", "Nanga Parbat",
               "Shishapangma"]
)

expectation_name_visibility = gx.expectations.ExpectColumnValuesToNotBeNull(
    column="name")

In [8]:
expectation_height = gx.expectations.ExpectColumnValuesToBeBetween(column="height", max_value=10000, min_value=7000
                                                                   )

expectation_height_visibility = gx.expectations.ExpectColumnValuesToNotBeNull(
    column="height"
)

In [9]:
expectation_country = gx.expectations.ExpectColumnDistinctValuesToBeInSet(
    column="country",
    value_set=["Pakistan", "India/Nepal", "Nepal", "India", "Pakistan", "China/Nepal", "China"]
)
expectation_city_visibility = gx.expectations.ExpectColumnValuesToNotBeNull(
    column="country")

### Creating ExpectationSuite and adding Expectations

In [10]:
expectation_suite_name = "temperature_data_suite"
expectation_suite_ref = gx.ExpectationSuite(name=expectation_suite_name)
expectation_suite = context.suites.add(expectation_suite_ref)

In [11]:
#Adding Expectation to ExpectationSuite    
expectation_suite.add_expectation(expectation_id)
expectation_suite.add_expectation(expectation_id_visibility)
expectation_suite.add_expectation(expectation_name_value)
expectation_suite.add_expectation(expectation_name_visibility)
expectation_suite.add_expectation(expectation_height)
expectation_suite.add_expectation(expectation_height_visibility)
expectation_suite.add_expectation(expectation_country)
expectation_suite.add_expectation(expectation_city_visibility)

ExpectColumnValuesToNotBeNull(id='e9a31c38-7b9f-4eeb-9e7d-c7df68ad90dd', meta=None, notes=None, result_format=<ResultFormat.BASIC: 'BASIC'>, description=None, catch_exceptions=True, rendered_content=None, windows=None, batch_id=None, column='country', mostly=1, row_condition=None, condition_parser=None)

### Creating Validation Definition

In [12]:
validation_def_name = "Peak_data_validation"

validation_definition_ref = gx.ValidationDefinition(data=batch_definition,
                                                    suite=expectation_suite,
                                                    name=validation_def_name)

In [13]:
validation_definition = context.validation_definitions.add(validation_definition_ref)

### Creating Checkpoint

In [14]:
checkpoint_name = "Peak_run_checkpoint"
checkpoint_to_add = gx.Checkpoint(name=checkpoint_name,
                                  validation_definitions=[validation_definition],
                                  actions=[],
                                  result_format="COMPLETE",  #Options : BOOLEAN_ONLY, BASIC, SUMMARY, COMPLETE
                                  )


In [15]:
checkpoint = context.checkpoints.add(checkpoint_to_add)


### Reading Data in Pandas DataFrame

In [16]:
data_df = pd.read_csv('sample_peaks_300.csv')
data_df.head()

,id,name,height,country
0,1,Everest,8975,India
1,2,Lhotse,7637,China/Nepal
2,3,K2,7221,India/Nepal
3,4,Nanga Parbat,8185,China/Nepal
4,5,Annapurna,7052,Pakistan


### Creating BatchParameter and running Validation

In [17]:
batch_parameters = {"dataframe": data_df}

In [18]:
validation_results = checkpoint.run(batch_parameters=batch_parameters)

Calculating Metrics:   0%|          | 0/35 [00:00<?, ?it/s]

### Displaying Validation Result

In [19]:
print(validation_results)

run_id={"run_name": null, "run_time": "2025-06-18T12:49:10.412346+06:00"} run_results={ValidationResultIdentifier::temperature_data_suite/__none__/20250618T064910.412346Z/sample_peaks_300-Peak_entity_asset: {
  "success": false,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_be_unique",
        "kwargs": {
          "batch_id": "sample_peaks_300-Peak_entity_asset",
          "column": "id"
        },
        "meta": {},
        "id": "6231e687-8674-4a06-bb2d-b2fb06487c1e"
      },
      "result": {
        "element_count": 300,
        "unexpected_count": 0,
        "unexpected_percent": 0.0,
        "partial_unexpected_list": [],
        "missing_count": 0,
        "missing_percent": 0.0,
        "unexpected_percent_total": 0.0,
        "unexpected_percent_nonmissing": 0.0,
        "partial_unexpected_counts": [],
        "partial_unexpected_index_list": [],
        "unexpected_list": [],
        "unexpected_index_lis